1. 개인 환경 세팅시 셀1의 5번 !cp ~~ 경로 자기에 맞게 수정해야 함.
2. 런타임 환경 GPU 체크

In [ ]:
# 1. 시스템 패키지 설치 (오디오 처리를 위한 FFmpeg 필수)
!apt-get install -y ffmpeg

# 2. 라이브러리 설치
# 현재 Colab의 2.9.0 버전은 버그가 있으므로 2.5.1로 강제 다운그레이드
!pip install torch==2.5.1 torchaudio==2.5.1 soundfile demucs uvicorn fastapi python-multipart aiofiles pyngrok nest-asyncio

# 3. 구글 드라이브 마운트
from google.colab import drive
import os

drive.mount('/content/drive')

# 4. 필수 폴더 생성
!mkdir -p model
!mkdir -p static
!mkdir -p static_results

# =================================================================
# *** 5. 모델 파일 복사 (경로는 마운트된 파일 경로에 맞게 수정) 후 주석 해제 ***

#!cp "/content/drive/MyDrive/AIX_DL_FINAL_PROJECT/HYPark/MyWebDemucs/model/checkpoint.th" ./model/
#!cp "/content/drive/MyDrive/AIX_DL_FINAL_PROJECT/HYPark/MyWebDemucs/model/model2_mastering.pth" ./model/


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 96.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 74.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 

In [ ]:
# 5. index.html 파일 저장
%%writefile static/index.html
<!DOCTYPE html>
<html lang="ko">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>AI 스마트 믹서</title>
    <style>
        body { font-family: 'Segoe UI', sans-serif; text-align: center; padding: 20px; background-color: #121212; color: #e0e0e0; }
        .container { background: #1e1e1e; padding: 30px; border-radius: 15px; max-width: 800px; margin: auto; box-shadow: 0 10px 30px rgba(0,0,0,0.5); }
        h1 { color: #00d2ff; }

        .upload-section { border: 2px dashed #444; padding: 20px; border-radius: 10px; margin-bottom: 20px; }
        .btn { background: #007bff; color: white; border: none; padding: 10px 20px; border-radius: 5px; cursor: pointer; font-size: 16px; transition: 0.2s; }
        .btn:hover:not(:disabled) { background: #0056b3; }
        .btn:disabled { background: #444; cursor: not-allowed; }

        /* 로딩 스피너 */
        .spinner { border: 4px solid #f3f3f3; border-top: 4px solid #00d2ff; border-radius: 50%; width: 24px; height: 24px; animation: spin 1s linear infinite; display: inline-block; margin-right: 10px;}
        @keyframes spin { 0% { transform: rotate(0deg); } 100% { transform: rotate(360deg); } }

        /* 트랙 레이아웃 */
        .track-row {
            background: #2c2c2c; margin: 15px 0; padding: 15px; border-radius: 10px;
            display: flex; flex-wrap: wrap; align-items: center; justify-content: space-between;
            border-left: 5px solid #555;
        }
        .track-info { flex: 1; text-align: left; min-width: 150px; }
        .track-name { font-size: 18px; font-weight: bold; display: block; margin-bottom: 5px; text-transform: uppercase;}
        .eq-controls { flex: 2; display: flex; gap: 15px; justify-content: center; min-width: 300px; }

        /* 슬라이더 스타일 */
        .slider-group { text-align: center; }
        .slider-group label { display: block; font-size: 12px; color: #888; margin-bottom: 2px; }
        input[type=range] { width: 80px; accent-color: #00d2ff; cursor: pointer; }
        .val-display { font-size: 12px; font-weight: bold; color: #00d2ff; }

        audio { width: 100%; margin-top: 10px; height: 30px; }

        .master-controls { margin-top: 20px; padding-top: 20px; border-top: 1px solid #444; }
        .btn-green { background: #28a745; }
        .btn-red { background: #dc3545; }

        /* 트랙별 색상 */
        .t-vocals { border-color: #ff6b6b; color: #ff6b6b; }
        .t-drums { border-color: #4ecdc4; color: #4ecdc4; }
        .t-bass { border-color: #ffe66d; color: #ffe66d; }
        .t-other { border-color: #1a535c; color: #1a535c; }
    </style>
</head>
<body>
    <div class="container">
        <h1>🎛️ AI 스마트 믹서</h1>
        <p>AI가 제안한 EQ를 확인하고, 슬라이더로 직접 톤을 잡아보세요.</p>

        <!-- 메시지 및 로딩 상태 -->
        <div id="status" style="margin-top: 10px; color: #ffcc00; font-weight: bold;"></div>

        <form id="upload-form" class="upload-section">
            <input type="file" id="audioFile" name="file" accept=".mp3, .wav, .ogg" required>

            <div style="margin-top: 15px; text-align: left;">
                <input type="checkbox" id="use-ai-eq" name="use_ai_eq" checked>
                <label for="use-ai-eq" style="color: #bbb;">AI EQ 예측 사용 (MasteringAI)</label>
            </div>

            <button type="submit" id="uploadBtn" class="btn" style="margin-top: 15px;">업로드 및 분석</button>
        </form>

        <div id="mixer-area" style="display: none;">
            <div class="master-controls">
                <button class="btn btn-green" onclick="playAll()">▶ 전체 재생</button>
                <button class="btn btn-red" onclick="stopAll()">⏹ 전체 정지</button>
            </div>
            <div id="tracks-container"></div>
        </div>
    </div>

    <script>
        const API_URL = '/upload';

        const uploadForm = document.getElementById('upload-form');
        const audioFile = document.getElementById('audioFile');
        const uploadBtn = document.getElementById('uploadBtn');
        const statusDisplay = document.getElementById('status');
        const mixerArea = document.getElementById('mixer-area');
        const tracksContainer = document.getElementById('tracks-container');

        // Web Audio API 컨텍스트 (브라우저의 오디오 엔진)
        let audioCtx;
        const tracks = {}; // 트랙별 노드 저장소 { vocals: { low, mid, high } }

        // ====== 1. API 통신 및 상태 관리 ======

        uploadForm.addEventListener('submit', async function(e) {
            e.preventDefault();
            const file = audioFile.files[0];

            uploadBtn.disabled = true;
            statusDisplay.innerHTML = '<div class="spinner"></div> ⏳ 업로드 및 AI 분석 중... (오래 걸릴 수 있습니다)';
            mixerArea.style.display = 'none';
            tracksContainer.innerHTML = '';

            const formData = new FormData(uploadForm); // 폼의 모든 필드 (file, use_ai_eq)를 자동으로 포함

            try {
                const res = await fetch(API_URL, { method: 'POST', body: formData });
                const data = await res.json();

                if (res.ok && data.status === 'success') {
                    statusDisplay.innerText = "✅ 준비 완료! AI 추천 EQ가 설정되었습니다.";
                    mixerArea.style.display = 'block';
                    initMixer(data);
                } else {
                    const errorDetail = data.detail || '알 수 없는 서버 오류';
                    statusDisplay.innerHTML = `❌ 에러: ${errorDetail}`;
                    console.error("FastAPI Error:", data);
                }
            } catch (e) {
                console.error("통신 에러:", e);
                statusDisplay.innerHTML = `❌ 통신 에러: 서버에 연결할 수 없습니다.`;
            } finally {
                uploadBtn.disabled = false;
            }
        });

        // ====== 2. Web Audio API 초기화 및 연결 ======

        function initMixer(data) {
            tracksContainer.innerHTML = "";

            // Web Audio API 시작
            if (!audioCtx) audioCtx = new (window.AudioContext || window.webkitAudioContext)();

            // Web Audio API는 트랙별로 한 번씩만 설정해야 합니다.
            tracks.length = 0;

            const stems = ['vocals', 'drums', 'bass', 'other'];

            stems.forEach(stem => {
                if (!data.urls[stem]) return;

                const eqVals = data.eq_info[stem] || { low: 0, mid: 0, high: 0 };

                // 1. HTML UI 생성
                const div = document.createElement('div');
                div.className = `track-row t-${stem}`;
                div.innerHTML = `
                    <div class="track-info">
                        <span class="track-name">${stem}</span>
                        <small>AI 추천: L:${eqVals.low} / M:${eqVals.mid} / H:${eqVals.high}</small>
                    </div>
                    <div class="eq-controls">
                        ${createSliderHTML(stem, 'low', 'Low (100Hz)', eqVals.low)}
                        ${createSliderHTML(stem, 'mid', 'Mid (1kHz)', eqVals.mid)}
                        ${createSliderHTML(stem, 'high', 'High (10kHz)', eqVals.high)}
                    </div>
                    <!-- controls 속성이 필요함 -->
                    <audio id="audio-${stem}" src="${data.urls[stem]}" controls crossOrigin="anonymous"></audio>
                `;
                tracksContainer.appendChild(div);

                // 2. Web Audio API 연결
                setupAudioNodes(stem, eqVals);
            });
        }

        function createSliderHTML(stem, band, label, val) {
            // val을 0.1 단위로 고정
            const displayVal = parseFloat(val).toFixed(1);
            return `
                <div class="slider-group">
                    <label>${label}</label>
                    <input type="range" id="sl-${stem}-${band}" min="-15" max="15" step="0.1" value="${displayVal}"
                        oninput="updateEQ('${stem}', '${band}', this.value)">
                    <div class="val-display" id="val-${stem}-${band}">${displayVal}dB</div>
                </div>
            `;
        }

        function setupAudioNodes(stem, initialEq) {
            const audioEl = document.getElementById(`audio-${stem}`);

            // 미디어 소스 생성
            const source = audioCtx.createMediaElementSource(audioEl);

            // 필터 생성: Low Shelf (100Hz), Peaking (1000Hz), High Shelf (10000Hz)
            const lowFilter = audioCtx.createBiquadFilter();
            lowFilter.type = "lowshelf";
            lowFilter.frequency.value = 100;
            lowFilter.gain.value = initialEq.low;

            const midFilter = audioCtx.createBiquadFilter();
            midFilter.type = "peaking";
            midFilter.frequency.value = 1000;
            midFilter.Q.value = 1.0;
            midFilter.gain.value = initialEq.mid;

            const highFilter = audioCtx.createBiquadFilter();
            highFilter.type = "highshelf";
            highFilter.frequency.value = 10000;
            highFilter.gain.value = initialEq.high;

            // 연결: 소스 -> Low -> Mid -> High -> 스피커
            source.connect(lowFilter);
            lowFilter.connect(midFilter);
            midFilter.connect(highFilter);
            highFilter.connect(audioCtx.destination);

            // 참조 저장 (나중에 업데이트용)
            tracks[stem] = { low: lowFilter, mid: midFilter, high: highFilter };

            // 오디오 재생 시 AudioContext가 suspended 상태면 깨워줌
            audioEl.onplay = () => { if (audioCtx.state === 'suspended') audioCtx.resume(); };
        }

        // ====== 3. 컨트롤 함수 (글로벌) ======

        // 슬라이더 조작 시 실행되는 함수
        window.updateEQ = function(stem, band, value) {
            const floatVal = parseFloat(value);
            // 화면 숫자 업데이트
            document.getElementById(`val-${stem}-${band}`).innerText = floatVal.toFixed(1) + "dB";

            // 실제 오디오 필터 업데이트
            if (tracks[stem]) {
                tracks[stem][band].gain.value = floatVal;
            }
        };

        window.playAll = function() {
            // 재생 시 AudioContext resume 보장
            if (audioCtx && audioCtx.state === 'suspended') audioCtx.resume();
            document.querySelectorAll('audio').forEach(a => { a.currentTime = 0; a.play(); });
        };
        window.stopAll = function() {
            document.querySelectorAll('audio').forEach(a => { a.pause(); a.currentTime = 0; });
        };
    </script>
</body>
</html>

Overwriting static/index.html


In [ ]:
# Colab에서 main.py 파일 생성
%%writefile main.py
import os
import shutil
import subprocess
import uuid
import torch
import torch.nn as nn
import torchaudio
import torchaudio.transforms as T
import asyncio
import re
from fastapi import FastAPI, UploadFile, File, Form, HTTPException
from fastapi.responses import JSONResponse, FileResponse
from fastapi.staticfiles import StaticFiles
from urllib.parse import quote
from concurrent.futures import ThreadPoolExecutor

# =================================================================
# [설정] 경로 및 환경 변수
# =================================================================
# 모델 파일 경로 (셀1에서 복사한 경로와 일치해야 함)
EQ_MODEL_PATH = os.path.join("model", "model2_mastering.pth")
RESULT_DIR = "static_results"
os.makedirs(RESULT_DIR, exist_ok=True)

app = FastAPI()
app.mount("/static", StaticFiles(directory="static"), name="static")
app.mount("/results", StaticFiles(directory=RESULT_DIR), name="results")

# GPU 사용 가능 여부 확인
device = "cuda" if torch.cuda.is_available() else "cpu"

# =================================================================
# [중요] 모델 클래스는 반드시 함수 밖(Global Scope)에 있어야 pickle 에러가 안 남
# =================================================================
class MasteringAI(nn.Module):
    def __init__(self):
        super().__init__()
        self.to_spec = T.MelSpectrogram(sample_rate=22050, n_mels=64)
        self.to_db = T.AmplitudeToDB()
        self.cnn = nn.Sequential(
            nn.Conv2d(1, 16, 3, padding=1), nn.BatchNorm2d(16), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(16, 32, 3, padding=1), nn.BatchNorm2d(32), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1), nn.BatchNorm2d(64), nn.ReLU(), nn.MaxPool2d(2),
            nn.AdaptiveAvgPool2d((4, 4))
        )
        self.head = nn.Sequential(
            nn.Flatten(), nn.Linear(64 * 4 * 4, 128), nn.ReLU(), nn.Linear(128, 3)
        )
    def forward(self, x):
        spec = self.to_db(self.to_spec(x))
        return self.head(self.cnn(spec.unsqueeze(1)))

# 전역 모델 변수
ai_engineer = None

@app.on_event("startup")
def load_models():
    global ai_engineer
    print("=== 서버 시작: 모델 로딩 ===")

    # MasteringAI (EQ 예측 모델) 로드
    if os.path.exists(EQ_MODEL_PATH):
        try:
            ai_engineer = MasteringAI().to(device)
            # map_location을 사용하여 GPU/CPU 호환성 확보
            ai_engineer.load_state_dict(torch.load(EQ_MODEL_PATH, map_location=device))
            ai_engineer.eval()
            print("✅ AI EQ 예측 모델 로드 완료")
        except Exception as e:
            print(f"⚠️ EQ 모델 로드 실패: {e}")
    else:
        print(f"⚠️ 경고: {EQ_MODEL_PATH} 파일을 찾을 수 없습니다.")

# =================================================================
# [유틸 함수] 파일명 안전하게 만들기 (werkzeug 대체)
# =================================================================
def secure_filename(filename):
    filename = re.sub(r'[^a-zA-Z0-9_.-]', '_', filename)
    return filename

# =================================================================
# [함수] EQ 값만 예측해서 반환
# =================================================================
def predict_eq_values(file_path):
    if ai_engineer is None: return None
    try:
        waveform, sr = torchaudio.load(file_path)
        if sr != 22050:
            waveform = T.Resample(sr, 22050, dtype=waveform.dtype)(waveform)

        waveform = torch.mean(waveform, dim=0, keepdim=True)
        duration_samples = 3 * 22050

        if waveform.shape[1] > duration_samples:
            input_tensor = waveform[:, :duration_samples]
        else:
            input_tensor = torch.nn.functional.pad(waveform, (0, duration_samples - waveform.shape[1]))

        with torch.no_grad():
            prediction = ai_engineer(input_tensor.to(device))

        vals = prediction[0].cpu().numpy()
        return {
            "low": round(float(vals[0]), 1),
            "mid": round(float(vals[1]), 1),
            "high": round(float(vals[2]), 1)
        }
    except Exception as e:
        print(f"❌ EQ 예측 실패 ({file_path}): {e}")
        return None

# =================================================================
# [함수] Demucs 실행 (subprocess 사용이 가장 안정적임)
# =================================================================
def run_demucs_separation(input_path, save_folder, track_name):
    # Demucs 라이브러리는 CLI 명령어로 실행하는 것이 의존성 문제 없이 가장 확실함
    command = [
        "demucs",
        "-n", "htdemucs",  # 또는 'htdemucs_ft' (속도/품질에 따라 선택, 기본값 추천)
        "-o", save_folder,
        input_path
    ]

    # GPU가 없으면 CPU 강제 옵션 추가
    if device == "cpu":
        command.extend(["-d", "cpu"])

    print(f"Executing: {' '.join(command)}")
    result = subprocess.run(command, capture_output=True, text=True)

    if result.returncode != 0:
        print(f"Demucs Error:\n{result.stderr}")
        raise RuntimeError(f"음원 분리 실패: {result.stderr.splitlines()[-1] if result.stderr else 'Unknown Error'}")

# =================================================================
# [라우터]
# =================================================================
@app.get("/")
async def read_root():
    return FileResponse('static/index.html')

@app.post("/upload")
async def upload_file(file: UploadFile = File(...), use_ai_eq: bool = Form(True)):
    filename = secure_filename(file.filename)
    task_id = str(uuid.uuid4())

    # 폴더 구조: static_results / task_id / 원본파일
    save_folder = os.path.join(RESULT_DIR, task_id)
    os.makedirs(save_folder, exist_ok=True)
    input_path = os.path.join(save_folder, filename)

    try:
        with open(input_path, "wb") as buffer:
            shutil.copyfileobj(file.file, buffer)
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"파일 저장 중 오류: {e}")

    track_name_no_ext = os.path.splitext(filename)[0]

    print(f"[{track_name_no_ext}] 음원 분리 작업 시작...")
    try:
        # 비동기 실행
        await asyncio.to_thread(run_demucs_separation, input_path, save_folder, track_name_no_ext)
    except RuntimeError as e:
        raise HTTPException(status_code=500, detail=str(e))
    finally:
        if os.path.exists(input_path):
            os.remove(input_path) # 원본 삭제하여 용량 확보

    # Demucs 출력 폴더 찾기 (htdemucs 모델 기준)
    # 기본 구조: output_dir / htdemucs / track_name / stems...
    target_dir = os.path.join(save_folder, "htdemucs", track_name_no_ext)

    # 폴더명을 못 찾을 경우(특수문자 등)를 대비해 검색
    if not os.path.exists(target_dir):
        base_model_dir = os.path.join(save_folder, "htdemucs")
        if os.path.exists(base_model_dir):
            subfolders = [f.path for f in os.scandir(base_model_dir) if f.is_dir()]
            if subfolders: target_dir = subfolders[0]

    if not os.path.exists(target_dir):
        raise HTTPException(status_code=500, detail="음원 분리 결과 폴더를 찾을 수 없습니다.")

    stems = ["vocals.wav", "drums.wav", "bass.wav", "other.wav"]
    eq_results = {}

    # AI EQ 예측
    if use_ai_eq and ai_engineer is not None:
        for stem in stems:
            stem_path = os.path.join(target_dir, stem)
            if os.path.exists(stem_path):
                res = predict_eq_values(stem_path)
                if res:
                    eq_results[stem.replace(".wav", "")] = res

    # URL 반환
    response_data = {"status": "success", "track_name": track_name_no_ext, "urls": {}, "eq_info": eq_results, "task_id": task_id}
    abs_result_dir = os.path.abspath(RESULT_DIR)

    for stem in stems:
        full_path = os.path.join(target_dir, stem)
        if os.path.exists(full_path):
            # 절대 경로를 상대 경로로 변환 후 URL 인코딩
            rel_path = os.path.relpath(full_path, abs_result_dir).replace("\\", "/")
            # URL에 안전하게 인코딩 (공백 등 처리)
            web_path = f"/results/{quote(rel_path)}"
            response_data["urls"][stem.replace(".wav", "")] = web_path

    print(f"[{track_name_no_ext}] 완료.")
    return JSONResponse(content=response_data)

Overwriting main.py


In [4]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn
import asyncio
from getpass import getpass # 비밀번호 입력창 라이브러리

nest_asyncio.apply()

print("="*50)
print("Ngrok Authtoken을 입력하세요 (입력 시 글자가 안 보일 수 있음)")
print("발급 주소: https://dashboard.ngrok.com/get-started/your-authtoken")
print("="*50)

# 1. 입력창을 띄워서 토큰을 받음
token = getpass("Ngrok Token 입력: ")
ngrok.set_auth_token(token)

try:
    ngrok.kill()
    ngrok_tunnel = ngrok.connect(8000)
    print(f"\n🚀 AI 스마트 믹서 접속 주소: {ngrok_tunnel.public_url}\n")
except Exception as e:
    print(f"❌ Ngrok 연결 실패: {e}")

print("⏳ 서버 시작 중...")
config = uvicorn.Config("main:app", host="0.0.0.0", port=8000, log_level="info")
server = uvicorn.Server(config)
await server.serve()

Ngrok Authtoken을 입력하세요 (입력 시 글자가 안 보일 수 있음)
발급 주소: https://dashboard.ngrok.com/get-started/your-authtoken

🚀 AI 스마트 믹서 접속 주소: https://unweeded-uninceptively-reid.ngrok-free.dev

⏳ 서버 시작 중...


INFO:     Started server process [9426]
INFO:     Waiting for application startup.
/content/main.py:66: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ai_engineer.load_state_

=== 서버 시작: 모델 로딩 ===
✅ AI EQ 예측 모델 로드 완료
INFO:     121.169.185.235:0 - "GET / HTTP/1.1" 200 OK
[Brown_Eyed_Girls_-_Must_Have_Love__mp3.pm_] 음원 분리 작업 시작...
Executing: demucs -n htdemucs -o static_results/e1bdc130-386c-46e8-b9b8-d5b76f814cc3 static_results/e1bdc130-386c-46e8-b9b8-d5b76f814cc3/Brown_Eyed_Girls_-_Must_Have_Love__mp3.pm_.mp3
[Brown_Eyed_Girls_-_Must_Have_Love__mp3.pm_] 완료.
INFO:     121.169.185.235:0 - "POST /upload HTTP/1.1" 200 OK
INFO:     121.169.185.235:0 - "GET /results/e1bdc130-386c-46e8-b9b8-d5b76f814cc3/htdemucs/Brown_Eyed_Girls_-_Must_Have_Love__mp3.pm_/vocals.wav HTTP/1.1" 206 Partial Content
INFO:     121.169.185.235:0 - "GET /results/e1bdc130-386c-46e8-b9b8-d5b76f814cc3/htdemucs/Brown_Eyed_Girls_-_Must_Have_Love__mp3.pm_/drums.wav HTTP/1.1" 206 Partial Content
INFO:     121.169.185.235:0 - "GET /results/e1bdc130-386c-46e8-b9b8-d5b76f814cc3/htdemucs/Brown_Eyed_Girls_-_Must_Have_Love__mp3.pm_/bass.wav HTTP/1.1" 206 Partial Content
INFO:     121.169.185.235:0 - "G

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [9426]
